In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [ ]:
import json
from glob import glob

def jsons_to_df(folder):
    imgs = glob(f"{folder}/*.json")
    
    df = []
    for i in imgs:
        i = json.load(open(i, 'r'))
        df.append(pd.DataFrame({
            'image': i['imagePath'],
            'bbox': [[int(j) for j in i['shapes'][0]['points'][1]] + [int(j) for j in i['shapes'][0]['points'][0]]],
            'height': i['imageHeight'], 
            'width': i['imageWidth']
        }))
        
    return pd.concat(df).reset_index(drop=True)
        
    
train_df = jsons_to_df('../input/plant2021-bboxes')

train_df

In [ ]:
train_df.head()

In [ ]:
train_df['width'].unique(), train_df['height'].unique()

In [ ]:
for g in train_df.groupby('image'):
    b = g[1]['bbox'].values
    print(type(b),b)
    break

In [ ]:
total_data = []
for g in tqdm_notebook(train_df.groupby('image')):
    data = {}
    data['filename'] = g[0]
    data['bbox'] = g[1]['bbox'].values[0]
    data['height'] = g[1]['height'].values[0]
    data['width'] = g[1]['width'].values[0]
    total_data.append(data)

In [ ]:
len(total_data)

In [ ]:
total_data[0]

In [ ]:
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
# install detectron2:
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import random

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
!wget https://i.ytimg.com/vi/dcXMf6rmafs/maxresdefault.jpg  -O input.jpg
im = cv2.imread("./input.jpg")

import matplotlib.pyplot as plt

plt.imshow(im)

In [ ]:
!wget https://raw.githubusercontent.com/facebookresearch/detectron2/master/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml

In [ ]:
!wget https://raw.githubusercontent.com/facebookresearch/detectron2/master/configs/Base-RCNN-FPN.yaml

In [ ]:
cfg = get_cfg()
#cfg.merge_from_file("mask_rcnn_R_50_FPN_3x.yaml")
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  

#"detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"

In [ ]:
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
outputs

In [ ]:
%matplotlib inline

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
v

In [ ]:
plt.figure(figsize=(20, 15))
plt.imshow(v.get_image()[:, :, ::-1])

In [ ]:
total_data[0]

In [ ]:
import os
import numpy as np

from detectron2.structures import BoxMode

def get_dicts(total_data):
    
    dataset_dicts = []
    for idx, v in enumerate(total_data):
        record = {}
        
        filename = os.path.join('../input/plant2021-downscaled-images-dataset/', v["filename"])
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = v['height']
        record["width"] = v['width']
            
        record["annotations"] = [{
                'bbox': v['bbox'],
                'bbox_mode': BoxMode.XYXY_ABS,
                'category_id':0,
            }]
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog

index = int(0.9 * len(total_data))
train_data = total_data[:index]
val_data = total_data[index:]

folders = ['train', 'val']
for i, d in enumerate([train_data,val_data]):
    DatasetCatalog.register("plant_" + folders[i], lambda d=d: get_dicts(d))
    MetadataCatalog.get("plant_" + folders[i]).set(thing_classes=["plant"])

In [ ]:
len(val_data)

In [ ]:
plant_metadata = MetadataCatalog.get("plant_train")

In [ ]:
train_data[0]['bbox'][0]

In [ ]:
dataset_dicts = get_dicts(train_data)
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plant_metadata, scale=1)
    vis = visualizer.draw_dataset_dict(d)
    plt.figure(figsize=(15,15))
    plt.imshow(vis.get_image()[:,:,::-1])
    plt.show()

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("plant_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.0003  
cfg.SOLVER.MAX_ITER = 400   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 12   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   
cfg.DATASETS.TEST = ("plant_val", )
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_dicts(val_data)
for d in dataset_dicts:    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=plant_metadata, 
                   scale=0.8, 
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(v.get_image()[:,:,::-1])
    plt.show()

In [ ]:
im = cv2.imread('../input/plant2021-downscaled-images-dataset/832d2ee485c5ff11.jpg')
outputs = predictor(im)

In [ ]:
outputs

In [ ]:
v = Visualizer(im[:, :, ::-1],
                   metadata=plant_metadata, 
                   scale=0.8, 
    )
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(v.get_image()[:,:,::-1])
plt.show()

In [ ]:
box = outputs["instances"].get_fields()['pred_boxes'].tensor.cpu()[0]
box

In [ ]:
box = box.long()
box

In [ ]:
im.shape

In [ ]:
plt.imshow(im[85:527, 18:716])

In [ ]:
def crop_image(img):
    outputs = predictor(img)
    box = outputs["instances"].get_fields()['pred_boxes'].tensor.cpu()[0].long()
    return im[box[1]:box[3], box[0]:box[2]]

In [ ]:
crop_image(im)

In [ ]:
from IPython.display import YouTubeVideo, display
video = YouTubeVideo("lWqg010_90I", width=500) #ll8TgCZ0plk
display(video)

In [ ]:
# Install dependencies, download the video, and crop 5 seconds for processing
!pip install youtube-dl
!pip uninstall -y opencv-python opencv-contrib-python -y
!apt install python3-opencv -y
!youtube-dl https://www.youtube.com/watch?v=lWqg010_90I -f 22 -o video.mp4
!ffmpeg -i video.mp4 -t 00:00:06 -c:v copy video-clip.mp4